In [ ]:
%load_ext autoreload
%autoreload 2

# Processing Sensitivity Analysis Outputs

In [ ]:
import os
from pathlib import Path


ROOT = Path(os.getcwd()).parent
while not ROOT.joinpath(".git").exists():
    ROOT = ROOT.parent

# add the root to the python path
import sys
import dotenv
from omegaconf import OmegaConf

sys.path.append(str(ROOT))

# load the environment variables
dotenv.load_dotenv(ROOT.joinpath(".env"))

In [ ]:
import polars as pl

from sumo_pipelines.utils.config_helpers import (
    create_custom_resolvers,
)
from sumo_pipelines.utils.file_helpers import walk_directory

create_custom_resolvers()

## Load All Results Files

In [ ]:
regex_pattern = "tmp/SobolSensitivityAnalysisCars-*/**/results.parquet"

In [ ]:
results_df = []
for file_ in ROOT.glob(regex_pattern):
    print(file_)
    df = pl.read_parquet(file_)

    # read config
    config_file = file_.parent / "0" / "config.yaml"
    config = OmegaConf.load(config_file)

    df = df.with_columns(N=pl.lit(config.Blocks.SobolSequenceConfig.N))

    sample_df = pl.read_parquet(file_.parent / "sobol_sequence.parquet").with_row_count(
        "run_id"
    )

    results_df.append(df.join(sample_df, on="run_id"))

results_df = pl.concat(results_df)
repr_conf = config

In [ ]:
from src.sa_helpers.metrics import (
    SUMO_GASOLINE_GRAM_PER_LITER,
    SUMO_GASOLINE_GRAM_TO_JOULE,
)

results_df = results_df.with_columns(
    total_fuel_l_cropped=pl.col("total_energy")
    / SUMO_GASOLINE_GRAM_TO_JOULE
    / SUMO_GASOLINE_GRAM_PER_LITER
).with_columns(
    cropped_per_vehicle_fuel=pl.col("total_fuel_l_cropped")
    / pl.col("total_vehicles_emissions")
)

In [ ]:
results_df.filter(
    pl.col('N') == 2048
)

## Perform Sobol Analysis

In [ ]:
from SALib.analyze import sobol
from SALib.sample import sobol as sobol_sample
from sumo_pipelines.blocks.producers.config import SobolSequenceConfig

In [ ]:
problem = SobolSequenceConfig.build_sobol_dict(repr_conf.Blocks.SobolSequenceConfig)
second_order = repr_conf.Blocks.SobolSequenceConfig.calc_second_order


def get_sa_results(r_df, problem, column, second_order=False):
    return sobol.analyze(
        problem,
        r_df[column].to_numpy(),
        calc_second_order=second_order,
        seed=repr_conf.Metadata.random_seed,
    )


def get_sa_results_df(results_df, problem, columns, second_order=False):
    for n, n_df in results_df.group_by("N"):
        sa_results = {}
        for column in columns:
            sa_results[column] = get_sa_results(n_df, problem, column, second_order)

        sobol_df = pl.DataFrame(
            {
                "N": [n for _ in range(len(problem["names"]))],
                "names": problem["names"],
                **{
                    f"{col}_{sa_type}": sa_results[col][sa_type]
                    for col in columns
                    for sa_type in ["S1", "ST", "S1_conf", "ST_conf"]
                },
            }
        )

        yield sobol_df

In [ ]:
target_cols = set(results_df.columns).intersection(
    set(repr_conf.Pipeline.pipeline[0].consumers[8].config.keys())
) | set(
    [
        "cropped_per_vehicle_fuel",
        "total_fuel_l_cropped",
    ]
)

sobol_df = pl.concat(
    get_sa_results_df(results_df, problem, target_cols, second_order=second_order)
)

In [ ]:
from src.plotting.bar import plot_si_v_n

fig = plot_si_v_n(
    sobol_df.with_columns(
        pl.col("N") * (problem["num_vars"] + 2),
    ),
    plot_variables=["Accel", "Decel", "Tau", "speedFactor"],
    plot_columns=[
        "cropped_per_vehicle_fuel_ST",
        "average_travel_time_ST",
        "average_delay_ST",
        "delay_ratio_ST",
    ],
    pretty_columns=[
        "Fuel Consumption",
        "Travel Time",
        "Delay",
        "Delay Ratio",
    ],
)

img = fig.to_image(format="png", scale=2, width=800, height=800)
from IPython.display import Image

Image(img)
